In [64]:
import logging
import numpy as np

from naslib.search_spaces import SimpleCellSearchSpace
from naslib.defaults.trainer import Trainer
from naslib.utils.custom_dataset import CustomDataset
from naslib.optimizers import DARTSOptimizer, GDASOptimizer, RandomSearch
from naslib.search_spaces import NasBench301SearchSpace, SimpleCellSearchSpace
from naslib.utils import set_seed, setup_logger, get_config_from_args


from torch import nn
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from fvcore.common.config import CfgNode


In [68]:
class NasDataset(Dataset):
    def __init__(self, root_dir, name, queue, norm='min_max'):
        super().__init__()
        assert queue in ['train', 'test', 'val']

        self.root_dir = root_dir 
        self.name = name
        self.type = queue

        x_num = f'X_num_{queue}.npy'
        x_bin = f'X_bin_{queue}.npy'
        y = f'Y_{queue}.npy'
        
        print(root_dir / name / x_num)
        self.x_num = np.load(root_dir / name / x_num)
        self.x_bin = np.load(root_dir / name / x_bin)
        self.y = np.load(root_dir / name / y)

        if norm == 'min_max':
            self.x_num = (self.x_num - np.min(self.x_num, axis=0)) / (np.max(self.x_num, axis=0) - np.min(self.x_num, axis=0))
            self.x_bin = (self.x_bin - np.min(self.x_bin, axis=0)) / (np.max(self.x_bin, axis=0) - np.min(self.x_bin, axis=0))

            self.y = (self.y - np.min(self.y, axis=0)) / (np.max(self.y, axis=0) - np.min(self.y, axis=0))

    def __getitem__(self, i):
        return np.concatenate((self.x_num[i], self.x_bin[i])), self.y[i]
        
    def __len__(self):
        return len(self.x_num)

root_dir = Path('/home/data/')
name = 'microsoft'
ds = NasDataset(root_dir, name, 'train')

/home/data/microsoft/X_num_train.npy


In [84]:
with open('/home/table_nas/test.yaml') as f:
    config = CfgNode.load_cfg(f)

In [85]:
class MyCustomDataset(CustomDataset):
    def __init__(self, config, mode="train"):
        super().__init__(config, mode)


    def get_transforms(self, config):
        return Compose([ToTensor()]), Compose([ToTensor()])


    def get_data(self, data, train_transform, valid_transform):
        train_data = ds
        test_data = ds

        return train_data, test_data
    
dataset = MyCustomDataset(config)
train_queue, valid_queue, test_queue, train_transform, valid_transform = dataset.get_loaders()
    

In [86]:
class NasTrainer(Trainer):
    @staticmethod
    def build_search_dataloaders(config):
        return train_queue, valid_queue, _

In [87]:
search_space = SimpleCellSearchSpace() 

optimizer = DARTSOptimizer(**config.search)
optimizer.adapt_search_space(search_space, config.dataset)

trainer = NasTrainer(optimizer, config)
trainer.search()  # Search for an architecture
trainer.evaluate()  # Evaluate the best architecture

[05/16 17:37:08 nl.defaults.trainer]: Beginning search
[05/16 17:37:08 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.000566, +0.001846, +0.000566, -0.000124, -0.000084, 1
-0.000696, +0.000604, -0.000189, +0.001544, +0.000095, 3
-0.000925, -0.002551, +0.000732, +0.000387, -0.000100, 2
+0.000618, +0.000118, -0.000135, -0.000130, +0.000580, 0
+0.000994, +0.001377, -0.000762, +0.001626, +0.000961, 3
+0.000047, -0.001012, +0.000006, +0.000497, -0.001528, 3
-0.001130, -0.000139, -0.001644, +0.000340, +0.000379, 4
+0.000324, -0.001272, +0.000243, -0.000727, +0.000888, 4
-0.001316, +0.001397, -0.000534, +0.000024, -0.001019, 1
+0.001596, -0.000234, +0.000971, -0.000739, +0.001240, 0


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 16, 3, 3], but got 2-dimensional input of size [64, 136] instead

AttributeError: search

In [46]:
    config.data

AttributeError: data

In [26]:
config

CfgNode({'dataset': 'cifar10', 'seed': 99, 'search_space': 'nasbench201', 'out_dir': 'run', 'optimizer': 'darts', 'search': CfgNode({'checkpoint_freq': 5, 'batch_size': 64, 'learning_rate': 0.025, 'learning_rate_min': 0.001, 'momentum': 0.9, 'weight_decay': 0.0003, 'epochs': 50, 'warm_start_epochs': 0, 'grad_clip': 5, 'train_portion': 0.5, 'data_size': 25000, 'cutout': False, 'cutout_length': 16, 'cutout_prob': 1.0, 'drop_path_prob': 0.0, 'unrolled': False, 'arch_learning_rate': 0.0003, 'arch_weight_decay': 0.001, 'output_weights': True, 'fidelity': 200, 'tau_max': 10, 'tau_min': 0.1, 'sample_size': 10, 'population_size': 100, 'num_init': 10, 'k': 10, 'num_ensemble': 3, 'acq_fn_type': 'its', 'acq_fn_optimization': 'mutation', 'encoding_type': 'path', 'num_arches_to_mutate': 2, 'max_mutations': 1, 'num_candidates': 100, 'predictor_type': 'var_sparse_gp', 'debug_predictor': False, 'seed': 99, 'gpu': None}), 'evaluation': CfgNode({'checkpoint_freq': 30, 'batch_size': 96, 'learning_rate': 

In [ ]:
q